<a href="https://colab.research.google.com/github/JamorMoussa/Build-GPT-From-Scratch/blob/main/Notebooks/Build_GPT_From_Scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a GPT from scratch

**GPT** stands for **Generative Pre-Trained Transformer**, a probabilistic system based on the **Transformer** architecture. This architecture was introduced in the renowned 2017 paper by Google, [Attention is All You Need](https://arxiv.org/pdf/1706.03762). The paper proposed the **Transformer** model for machine translation tasks.

This notebook contains notes for **Andrej Karpathy**'s tutorial, titled [Let's Build GPT: From Scratch, in Code, Spelled Out](https://www.youtube.com/watch?v=kCc8FmEb1nY) on his YouTube channel.